[View in Colaboratory](https://colab.research.google.com/github/lizhicq/Algorithms-Manual/blob/master/Ch01%20Binary%20Search.ipynb)

# Binary Search & Log N




*   Universal fitable binary search model 
*   How to avoid dead loop?
*   Corner case dealing



---


*  三步翻转法
*  递增矩阵找数
*  快速幂算法
*  辗转相除法

  

## 带check 函数的二分

### 437. Copy Books
Given n books and the ith book has A[i] pages. You are given k people to copy the n books.

n books list in a row and each person can claim a continous range of the n books. For example one copier can copy the books from ith to jth continously, but he can not copy the 1st book, 2nd book and 4th book (without 3rd book).

They start copying books at the same time and they all cost 1 minute to copy 1 page of a book. What's the best strategy to assign books so that the slowest copier can finish at earliest time?

Example
Given array A = [3,2,4], k = 2.

Return 5( First person spends 5 minutes to copy book 1 and book 2 and second person spends 4 minutes to copy book 3. )
```

In [0]:
from sys import maxsize
class Solution:
    """
    @param pages: an array of integers
    @param k: An integer
    @return: an integer
    """
    def copyBooks(self, pages, k):

        def check(mid): # if mid is large enough to make k people finish their work before mid
            page_left = num = 0 
            for page in pages:
                if page > mid:
                    return False
                if page > page_left:
                    page_left = mid 
                    num += 1 
                if page <= page_left:
                    page_left -= page
            return num <= k 
        
        start, end = 0, maxsize
        while start + 1 < end:
            mid = start + (end - start) / 2 
            if check(mid):
                end = mid
            else:
                start = mid
        
        if check(start): # start is large enough 
            return start
        return end

### 183. Wood Cut
Given n pieces of wood with length L[i] (integer array). Cut them into small pieces to guarantee you could have equal or more than k pieces with the same length. What is the longest length you can get from the n pieces of wood? Given L & k, return the maximum length of the small pieces.
```
Example
For L=[232, 124, 456], k=7, return 114.

Challenge
O(n log Len), where Len is the longest length of the wood.
```

In [0]:
class Solution:
    """
    @param L: Given n pieces of wood with length L[i]
    @param k: An integer
    @return: The maximum length of the small pieces
    """
    def woodCut(self, L, k):
        if L == []:
            return 0 
        start, end = 0, max(L)
        while start + 1 < end:
            mid = start + (end - start) // 2
            if self.check(mid, L, k):
                start = mid
            else:
                end = mid
        if self.check(end, L, k):
            return end
        return start

    def check(self, mid, L, k):# check if mid is small enough to make more than k woods
        num = 0
        for wood in L:
            num += wood // mid
        return num >= k

### Maximum Average Subarray II
Given an array with positive and negative numbers, find the maximum average subarray which length should be greater or equal to given length k.

```
Example
Given nums = [1, 12, -5, -6, 50, 3], k = 3

Return 15.667 // (-6 + 50 + 3) / 3 = 15.667
```

我们建立一个累加和数组sums，然后求出原数组中最小值赋给left，最大值赋给right，题目中说了误差是1e-5，所以我们的循环条件就是right比left大1e-5，然后我们算出来mid，定义一个minSum初始化为0，布尔型变量check，初始化为false。然后开始遍历数组，先更新累加和数组sums，注意这个累加和数组不是原始数字的累加，而是它们和mid相减的差值累加。我们的目标是找长度大于等于k的子数组的平均值大于mid，由于我们每个数组都减去了mid，那么就转换为找长度大于等于k的子数组的差累积值大于0。我们建立差值累加数组的意义就在于通过sums[i] - sums[j]来快速算出j和i位置中间数字之和，那么我们只要j和i中间正好差k个数字即可，然后minSum就是用来保存j位置之前的子数组差累积的最小值，所以当i >= k时，我们用sums[i - k]来更新minSum，这里的i - k就是j的位置，然后判断如果sums[i] - minSum > 0了，说明我们找到了一段长度大于等k的子数组平均值大于mid了，就可以更新left为mid了，我们标记check为true，并退出循环。在for循环外面，当check为true的时候，left更新为mid，否则right更新为mid，参见代码如下：

In [0]:
class Solution:
    """
    @param: nums: an array with positive and negative numbers
    @param: k: an integer
    @return: the maximum average
    """
    def maxAverage(self, nums, k):
        left = min(nums) * 1.0
        right = max(nums)
        eps = 1e-5
        csum = [0 for i in range(len(nums)+1)]
        min_pre = [0 for i in range(len(nums)+1)]
        
        def check(mid):
            csum[0] = 0.0
            min_pre[0] = 0.0
          
            for i in range(1, len(nums)+1):
                csum[i] = csum[i-1] + nums[i-1] - mid
                min_pre[i] = min(min_pre[i-1], csum[i])
            
            if i >= k and csum[i] - min_pre[i-k] >= 0:
                return True
            return False
        
        while left + eps < right:
            mid = left + (right -left)/2
            if check(mid):
                left = mid
            else:
                right = mid
        
        return left
      
if __name__ == "__main__":
    print Solution().maxAverage([1,2,3,4], 2)

3.4999961853


### Maximum Average Subarray III
Given an array with positive and negative numbers, find the maximum average subarray which length should be [k1, k2].

```
Example
Given nums = [1, 12, -5, -6, 50, 3], k = 3

Return 15.667 // (-6 + 50 + 3) / 3 = 15.667
```

### 600. Smallest Rectangle Enclosing Black Pixels
An image is represented by a binary matrix with 0 as a white pixel and 1 as a black pixel. The black pixels are connected, i.e., there is only one black region. Pixels are connected horizontally and vertically. Given the location (x, y) of one of the black pixels, return the area of the smallest (axis-aligned) rectangle that encloses all black pixels.
```
Example
For example, given the following image:

[
  "0010",
  "0110",
  "0100"
]
and x = 0, y = 2,
Return 6.
```

In [0]:
class Solution(object):
    # param image {List[List[str]]}  a binary matrix with '0' and '1'
    # param x, y {int} the location of one of the black pixels
    # return an integer

    def minArea(self, image, x, y):
        m = len(image)
        if m == 0:
            return 0
        n = len(image[0])
        if n == 0:
            return 0

        # search left
        start, end = 0, y
        while start + 1 < end:
            mid = start + (end - start) // 2
            if self.checkCol(image, mid):
                end = mid
            else:
                start = mid
        left = start if self.checkCol(image, start) else end
 
        # search right
        start, end = y, len(image[0]) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2
            if self.checkCol(image, mid):
                start = mid
            else:
                end = mid
        right = end if self.checkCol(image, end) else start

        start, end = 0, x
        while start + 1 < end:
            mid = start + (end - start) // 2
            if self.checkRow(image, mid):
                end = mid
            else:
                start = mid

        up = start if self.checkRow(image, start) else end

        start, end = x, len(image) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2
            if self.checkRow(image, mid):
                start = mid
            else:
                end = mid
        down = end if self.checkRow(image, end) else start
        
        return (right - left + 1) * (down - up + 1)

    def checkCol(self, image, col):
        for i in range(len(image)):
            if image[i][col] == '1':
                return True
        return False

    def checkRow(self, image, row):
        for j in range(len(image[0])):
            if image[row][j] == '1':
                return True
        return False

## LogN二分
### 快速幂算法
```
原本O(n)O(n)的问题，我们可以用O(logn)O(logn)复杂度的算法来解决。

递归版本的快速幂算法
int power(int x, int n) {
    if (n == 0) return 1;
    if (n % 2 == 0) {
        int tmp = power(x, n / 2);
        return tmp * tmp;
    } else {
        int tmp = power(x, n / 2);
        return tmp * tmp * x;
    }
}
非递归版本
int power(int x, int n) {
    int ans = 1, base = x;
    while (n != 0) {
        if (n % 2 == 1) {
            ans *= base;
        }
        base *= base;
        n = n / 2;
    }
    return ans;
}
非递归版本与递归版本原理相同，计算顺序略有不同。因为递归是从大问题进入，划分子问题然后层层返回再求解大问题。这里要从小问题开始，直接求解大问题。你可以打印出每次循环中 basebase 和 ansans 的值，来理清楚其中的算法思路。递归版本和非递归版本都应该熟练掌握，虽然递归版本更容易掌握和理解，且 lognlogn 的计算深度也不会导致 Stack Overflow。但是面试官是很有可能为了加大难度让你在用非递归的版本写一遍的。
```




### 428. Pow(x, n)
Implement pow(x, n).
```
Example
Pow(2.1, 3) = 9.261
Pow(0, 1) = 0
Pow(1, 0) = 1
Challenge
O(logn) time
```

In [0]:
class Solution:
    """
    @param: x: the base number, float
    @param: n: the power number, int
    @return: the results
    """
    def myPow(self, x, n):
        if n < 0:
            n = -n 
            x = 1 / x
        
        ans = 1
        tmp = x

        while n != 0:
            if n % 2 == 1:
                ans *= tmp
            tmp *= tmp
            n //= 2 
            
        return ans 

###  140. Fast Power
Calculate the an % b where a, b and n are all 32bit integers.
```
Example
For 231 % 3 = 2

For 1001000 % 1000 = 0

Challenge
O(logn)
```

In [0]:
class Solution:
    """
    @param a: A 32bit integer
    @param b: A 32bit integer
    @param n: A 32bit integer
    @return: An integer
    """
    def fastPower(self, a, b, n):
        
        ans = 1
        tmp = a
        
        while n != 0:
            if n % 2 == 1:
                ans = ans * tmp % b
            tmp = tmp * tmp % b
            n /= 2 
        ans = ans % b
        
        return ans


### 141. Sqrt(x)
Implement int sqrt(int x).

Compute and return the square root of x.
```
Example
sqrt(3) = 1

sqrt(4) = 2

sqrt(5) = 2

sqrt(10) = 3

Challenge
O(log(x))
```

In [0]:
class Solution:
    """
    @param x: An integer
    @return: The sqrt of x
    """
    def sqrt(self, x):
        start, end = 0.0, 65535
        eps = 1e-1
        while start + eps < end:
            mid = start + (end - start) / 2
            if mid * mid < x:
                start = mid
            else:
                end = mid
        return int(end)


### 845. Greatest Common Divisor (辗转相除法)


Given two numbers, number a and number b. Find the greatest common divisor of the given two numbers.
```
Example
Given a = 10, b = 15, return 5.
Given a = 15, b = 30, return 15.
```
辗转相除法， 又名欧几里德算法， 是求最大公约数的一种方法。它的具体做法是：用较大的数除以较小的数，再用除数除以出现的余数（第一余数），再用第一余数除以出现的余数（第二余数），如此反复，直到最后余数是0为止。如果是求两个数的最大公约数，那么最后的除数就是这两个数的最大公约数。
```
代码
public int gcd(int big, int small) {
    if (small != 0) {
        return gcd(small, big % small);
    } else {
        return big;
    }
}
```

In [0]:
class Solution(object):
    def gcd(self, big, small):
        if big < small:
            big, small = small, big
        if small != 0:
            return self.gcd(small, big % small)
        else:
            return big
        
        
if __name__ == "__main__":
    s = Solution()
    print s.gcd(10, 5)

5


### 235. Prime Factorization
Prime factorize a given integer.
```
Example
Given 10, return [2, 5].

Given 660, return [2, 2, 3, 5, 11].
```

In [0]:
class Solution:
    """
    @param num: An integer
    @return: an integer array
    """
    def primeFactorization(self, num):
        import math
        up = int(math.sqrt(num)) + 1
        prime = []
        f = [0 for x in range(up)]
        for i in range(2, up): # up >= 2
            if f[i] == 0:
                prime.append(i)
                for j in range(i*i, up, i):
                    f[j] = 1
        ret = []
        for a in prime:
            while num % a == 0:
                ret.append(a)
                num /= a
        if num != 1:
            ret.append(num)
        return ret

### 414. Divide Two Integers
Divide two integers without using multiplication, division and mod operator.

If it is overflow, return 2147483647
```
Example
Given dividend = 100 and divisor = 9, return 11.
```

In [0]:
class Solution(object):
    def divide(self, dividend, divisor):
        maxsize = 2147483647
        if divisor == 0:
            return maxsize
        if abs(dividend) < abs(divisor):
            return 0

        is_negtive = (dividend < 0 and divisor > 0
                     ) or (dividend > 0 and divisor < 0)
        a = abs(dividend)
        b = abs(divisor)

        ans = 0
        while a >= b:
            shift = 0
            while a >= (b << shift):
                shift += 1
            a -= b << (shift - 1)
            ans += 1 << (shift - 1)
        if is_negtive:
            ans = - ans
        if ans > maxsize:
            return maxsize
        return ans
    


if __name__ == "__main__":
    print Solution().divide(100, 2)

## 数组二分

### 458. Last Position of Target

Description
Find the last position of a target number in a sorted array. Return -1 if target does not exist.

Have you met this question in a real interview?  
Example
Given [1, 2, 2, 4, 5, 5].

For target = 2, return 2.

For target = 5, return 5.

For target = 6, return -1.

In [0]:
class Solution(object):
    def lastPosition(self, nums, target):
        if len(nums) == 0:
            return -1
        start, end = 0, len(nums) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if nums[mid] <= target:
                start = mid
            else:
                end = mid
        if nums[end] == target:
            return end
        elif nums[start] == target:
            return start
        else:
            return -1

### 460. Find K Closest Elements
Given a target number, a non-negative integer k and an integer array A sorted in ascending order, find the k closest numbers to target in A, sorted in ascending order by the difference between the number and target. Otherwise, sorted in ascending order by number if the difference is same.
```
Example
Given A = [1, 2, 3], target = 2 and k = 3, return [2, 1, 3].

Given A = [1, 4, 6, 8], target = 3 and k = 3, return [4, 1, 6].

Challenge
O(logn + k) time complexity.
```

In [0]:
class Solution:
    """
    @param A: an integer array
    @param target: An integer
    @param k: An integer
    @return: an integer array
    """

    def kClosestNumbers(self, A, target, k):
        start, end = 0, len(A) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2
            if A[mid] < target:
                start = mid
            else:
                end = mid
        res = []
        while len(res) != k:
            left_diff = target - A[start] if start >= 0 else None
            right_diff = A[end] - target if end < len(A) else None

            if left_diff is not None and right_diff is not None:
                if right_diff < left_diff:
                    res.append(A[end])
                    end = end + 1
                else:
                    res.append(A[start])
                    start = start - 1
            elif left_diff is None and right_diff is not None:
                res.append(A[end])
                end = end + 1
            elif left_diff is not None and right_diff is None:
                res.append(A[start])
                start = start - 1
            else:
                break  # K is larger than len(A)
        return res

### 585. Maximum Number in Mountain Sequence

Description
Given a mountain sequence of n integers which increase firstly and then decrease, find the mountain top.
```
Example
Given nums = [1, 2, 4, 8, 6, 3] return 8
Given nums = [10, 9, 8, 7], return 10
```

In [0]:
class Solution:
    """
    @param: nums: a mountain sequence which increase firstly and then decrease
    @return: then mountain top
    """
    def mountainSequence(self, nums):
        if len(nums) == 0: 
            return None
        if len(nums) == 1:
            return nums[0]
        if len(nums) == 2:
            return nums[0] if nums[0] > nums[1] else nums[1]
    
        start, end = 0, len(nums) - 1 
        
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if nums[mid] < nums[mid+1]:
                start = mid
            elif nums[mid] > nums[mid+1]:
                end = mid
        if nums[start] > nums[end]:
            return nums[start]
        elif nums[start] < nums[end]:
            return nums[end]
        else:
            return -1

### 74. First Bad Version
The code base version is an integer start from 1 to n. One day, someone committed a bad version in the code case, so it caused this version and the following versions are all failed in the unit tests. Find the first bad version.

You can call isBadVersion to help you determine which version is the first bad one. The details interface can be found in the code's annotation part.
```
Example
Given n = 5:

isBadVersion(3) -> false
isBadVersion(5) -> true
isBadVersion(4) -> true
Here we are 100% sure that the 4th version is the first bad version.

Challenge
You should call isBadVersion as few as possible.

```

In [0]:
class SVNRepo:
    @classmethod
    def isBadVersion(cls, id):
        pass
        # Run unit tests to check whether verison `id` is a bad version
        # return true if unit tests passed else false.
# You can use SVNRepo.isBadVersion(10) to check whether version 10 is a 
# bad version.

class Solution:
    """
    @param n: An integers.
    @return: An integer which is the first bad version.
    """
    def findFirstBadVersion(self, n):
        start, end = 1, n
        while start + 1 < end:
            mid = start + (end - start) // 2
            if SVNRepo.isBadVersion(mid):
                end = mid
            else:
                start = mid
        if SVNRepo.isBadVersion(start):
            return start
        return end

###75. Find Peak Element
There is an integer array which has the following features:
```
The numbers in adjacent positions are different.
A[0] < A[1] && A[A.length - 2] > A[A.length - 1].
We define a position P is a peak if:

A[P] > A[P-1] && A[P] > A[P+1]
Find a peak element in this array. Return the index of the peak.

Example
Given [1, 2, 1, 3, 4, 5, 7, 6]

Return index 1 (which is number 2) or 6 (which is number 7)

Challenge
Time complexity O(logN)
```

In [0]:
class Solution(object):
    def findPeak(self, nums):
        start, end = 0, len(nums) - 2 # prevent overlap
        while start + 1 < end:
            mid = start + (end - start) // 2

            if nums[mid] < nums[mid + 1]:
                 start = mid
            else:
                 end = mid
        if nums[start] < nums[end]:
            return end
        else:
            return start

### 447. Search in a Big Sorted Array
Given a big sorted array with positive integers sorted by ascending order. The array is so big so that you can not get the length of the whole array directly, and you can only access the kth number by ArrayReader.get(k) (or ArrayReader->get(k) for C++). Find the first index of a target number. Your algorithm should be in O(log k), where k is the first index of the target number.

Return -1, if the number doesn't exist in the array.
```
Example
Given [1, 3, 6, 9, 21, ...], and target = 3, return 1.

Given [1, 3, 6, 9, 21, ...], and target = 4, return -1.

Challenge
O(log k), k is the first index of the given target number.
```

In [0]:
class Solution(object):
    def searchBigSortedArray(self, reader, target):
        start, end = 0, 1
        while reader.get(end) < target:
            end = 2 * end + 1
        while start + 1 < end:
            mid = start + (end - start) // 2      
            if reader.get(mid) < target:
                start = mid
            elif reader.get(mid) >= target:
                end = mid
        if reader.get(start) == target:
            return start
        elif reader.get(end) == target:
            return end
        return -1 

### 159. Find Minimum in Rotated Sorted Array
Suppose a sorted array is rotated at some pivot unknown to you beforehand.

(i.e., 0 1 2 4 5 6 7 might become 4 5 6 7 0 1 2).

Find the minimum element.
```
Example
Given [4, 5, 6, 7, 0, 1, 2] return 0

```

In [0]:
class Solution(object):
    def findMin(self, nums) :
        if len(nums) == 0:
            return -1
        start, end = 0, len(nums) - 1 
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if nums[mid] > nums[-1]:
                start = mid
            else:
                end = mid
        if nums[start] < nums[end]:
            return nums[start]
        else:
            return nums[end]

### 62. Search in Rotated Sorted Array
Suppose a sorted array is rotated at some pivot unknown to you beforehand.

(i.e., 0 1 2 4 5 6 7 might become 4 5 6 7 0 1 2).

You are given a target value to search. If found in the array return its index, otherwise return -1.

You may assume no duplicate exists in the array.
```
Example
For [4, 5, 1, 2, 3] and target=1, return 2.

For [4, 5, 1, 2, 3] and target=0, return -1.

Challenge
O(logN) time

```

In [0]:
class Solution:
    """
    @param A : a list of integers
    @param target : an integer to be searched
    @return : an integer
    """
    def search(self, A, target):
        if A == None or len(A) == 0:
            return -1
        start, end = 0, len(A) - 1,
        while start + 1 < end:
            mid = start + (end - start) / 2
            if A[mid] == target:
                return mid
            if A[start] < A[mid]:
                if A[start] <= target <= A[mid]:
                    end = mid
                else:
                    start = mid
            else:
                if A[mid] <= target <= A[end]:
                    start = mid
                else:
                    end = mid
        if A[start] == target:
            return start
        if A[end] == target:
            return end
        return -1

### 462. Total Occurrence of Target
Given a target number and an integer array sorted in ascending order. Find the total number of occurrences of target in the array.
```
Example
Given [1, 3, 3, 4, 5] and target = 3, return 2.

Given [2, 2, 3, 4, 6] and target = 4, return 1.

Given [1, 2, 3, 4, 5] and target = 6, return 0.

Challenge
Time complexity in O(logn)
```

In [0]:
class Solution:
    # param {int[]} A an integer array sorted in ascending order
    # param {int} target an integer
    # return {int} an integer
    def totalOccurrence(self, A, target):
        # Write your code here
        if not A:
            return 0

        length = len(A)

        start = 0
        end = length - 1
        while start + 1 < end:
            mid = start + (end - start) / 2
            if A[mid] < target:
                start = mid
            else:
                end = mid
        if A[start] == target:
            head = start
        elif A[end] == target:
            head = end
        else:
            head = -1

        start = head
        end = length - 1
        while start + 1 < end:
            mid = start + (end - start) / 2
            if A[mid] <= target:
                start = mid
            else:
                end = mid
        if A[end] == target:
            tail = end
        elif A[start] == target:
            tail = start
        else:
            tail = -1

        if head >= 0 and tail >= 0:
            return tail - head + 1
        else:
            return 0

### 14. First Position of Target
For a given sorted array (ascending order) and a target number, find the first index of this number in O(log n) time complexity.

If the target number does not exist in the array, return -1.
```
Example
If the array is [1, 2, 3, 3, 4, 5, 10], for given target 3, return 2.

Challenge
If the count of numbers is bigger than 2^32, can your code work properly?
```

In [0]:
class Solution:
    # param nums: The integer array
    # param target: Target number to find
    # return the first position of target in nums, position start from 0 
    def binarySearch(self, nums, target):
        # write your code here
        left, right = 0, len(nums)
        while left + 1 < right :
            mid = (left + right) / 2
            if nums[mid] < target :
                left = mid
            else :
                right = mid
        if nums[left] == target :
            return left
        elif nums[right] == target :
            return right
        return -1;

### 28. Search a 2D Matrix
Write an efficient algorithm that searches for a value in an m x n matrix.

This matrix has the following properties:

Integers in each row are sorted from left to right.
The first integer of each row is greater than the last integer of the previous row.
```
Example
Consider the following matrix:

[
    [1, 3, 5, 7],
    [10, 11, 16, 20],
    [23, 30, 34, 50]
]
Given target = 3, return true.

Challenge
O(log(n) + log(m)) time
 
```

In [0]:
class Solution:
    """
    @param matrix, a list of lists of integers
    @param target, an integer
    @return a boolean, indicate whether matrix contains target
    """

    def searchMatrix(self, matrix, target):
        if len(matrix) == 0:
            return False
        row_start, row_end = 0, len(matrix) - 1
        
        while row_start + 1 < row_end:
            row_mid = row_start + (row_end - row_start) // 2
            if matrix[row_mid][0] == target:
                return True
            elif matrix[row_mid][0] < target:
                row_start = row_mid
            elif matrix[row_mid][0] > target:
                row_end = row_mid

        if self.searchRow(matrix[row_start], target) or self.searchRow(matrix[row_end], target):
            return True

        return False

    def searchRow(self, nums, target):
        start, end = 0, len(nums) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2
            if nums[mid] == target:
                return True
            elif nums[mid] < target:
                start = mid
            else:
                end = mid
        if nums[start] == target or nums[end] == target:
            return True
        return False

### 38. Search a 2D Matrix II
Write an efficient algorithm that searches for a value in an m x n matrix, return the occurrence of it.

This matrix has the following properties:

Integers in each row are sorted from left to right.
Integers in each column are sorted from up to bottom.
No duplicate integers in each row or column.
```
Example
Consider the following matrix:

[
  [1, 3, 5, 7],
  [2, 4, 7, 8],
  [3, 5, 9, 10]
]
Given target = 3, return 2.

Challenge
O(m+n) time and O(1) extra space
```

In [0]:
class Solution:
    """
    @param matrix: An list of lists of integers
    @param target: An integer you want to search in matrix
    @return: An integer indicates the total occurrence of target in the given matrix
    """
    def searchMatrix(self, matrix, target):
        if matrix == [] or matrix[0] == []:
            return 0
            
        row, column = len(matrix), len(matrix[0])
        i, j = row - 1, 0
        count = 0
        while i >= 0 and j < column:
            if matrix[i][j] == target:
                count += 1
                i -= 1
                j += 1
            elif matrix[i][j] < target:
                j += 1
            elif matrix[i][j] > target:
                i -= 1
        return count

### 61. Search for a Range
Given a sorted array of n integers, find the starting and ending position of a given target value.

If the target is not found in the array, return [-1, -1].
```
Example
Given [5, 7, 7, 8, 8, 10] and target value 8,
return [3, 4].

Challenge
O(log n) time.
```

In [0]:
class Solution:
    """
    @param A : a list of integers
    @param target : an integer to be searched
    @return : a list of length 2, [index1, index2]
    """
    def searchRange(self, A, target):
        try:
            pos = A.index(target)
        except:
            return [-1,-1]
        res = [pos]
        for i in range(pos + 1,len(A)):
            if A[pos] == A[pos + 1]:
                pos += 1
            else:
                break
        res.append(pos)
        return res
        
        

### 459. Closest Number in Sorted Array
Given a target number and an integer array A sorted in ascending order, find the index i in A such that A[i] is closest to the given target.

Return -1 if there is no element in the array.
```
Example
Given [1, 2, 3] and target = 2, return 1.

Given [1, 4, 6] and target = 3, return 1.

Given [1, 4, 6] and target = 5, return 1 or 2.

Given [1, 3, 3, 4] and target = 2, return 0 or 1 or 2.

Challenge
O(logn) time complexity.
```

In [0]:
class Solution:
    def closestNumber(self, A, target):
        nums = A
        start, end = 0, len(nums) - 1
        while start + 1 < end:
            mid = start + (end - start) // 2 
            if nums[mid] == target:
                return mid
            elif nums[mid] > target:
                end = mid
            elif nums[mid] < target:
                start = mid
        if A[end] - target < target - A[start]:
            return end
        else:
            return start


### 254. Drop Eggs
There is a building of n floors. If an egg drops from the k th floor or above, it will break. If it's dropped from any floor below, it will not break.

You're given two eggs, Find k while minimize the number of drops for the worst case. Return the number of drops in the worst case.
```
Example
Given n = 10, return 4.
Given n = 100, return 14.
```

In [0]:
import math
class Solution(object):
    def dropEggs(self, n):
        return int(math.ceil((math.sqrt(8*n+1)-1)/2))

### 584. Drop Eggs II
There is a building of n floors. If an egg drops from the k th floor or above, it will break. If it's dropped from any floor below, it will not break.

You're given m eggs, Find k while minimize the number of drops for the worst case. Return the number of drops in the worst case.

Example
Given m = 2, n = 100 return 14
Given m = 2, n = 36 return 8

In [0]:
class Solution(object):
    def dropEggs2(self, m, n):
        maxint = 65535
        f = [[maxint for j in range(n+1)]
                      for i in range(m+1)]
        f[1][1] = 1
        for i in range(1, m+1):
            f[i][1] = 1

        for j in range(1, n+1):
            f[1][j] = j

        for i in range(2, m+1):
            for j in range(2, n+1):
                for k in range(2, j):
                    f[i][j] = min(f[i][j], 1 + max(f[i-1][k-1], f[i][j-k]))
                    #print i, j, k, f[i][j]
        return f[m][n]